## Motor-selective neurons vs. heading direction neurons
In this notebook, we compare the activity/location of neurons that are left/right motion selective, and activity of heading direction selective neurons.

#### TODO
 - [ ] make map including all fish after the morphing 

In [ ]:
%matplotlib widget
from pathlib import Path

import numpy as np
import pandas as pd
from bouter.utilities import crop
from lotr import A_FISH, LotrExperiment, dataset_folders
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
# path = # master_path / "210314_f1" / "210314_f1_natmov"
exp = LotrExperiment(A_FISH)

In [ ]:
bouts_df = exp.bouts_df
regr_df = exp.motor_regressors

THR = 0.4
OFF = 0.45
directions = "lf", "rt"
bout_sel = dict(
    rt=(regr_df["right_1"] > (regr_df["left_1"] + OFF)) & (regr_df["right_1"] > THR),
    lf=(regr_df["left_1"] > (regr_df["right_1"] + OFF)) & (regr_df["left_1"] > THR),
)

In [ ]:
concat = exp.traces[:, regr_df["all_1"]>0.7]

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2).fit(concat.T)
pcaed = pca.transform(concat.T)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.figure()
plt.scatter(pcaed[:, 0], pcaed[:, 1])

In [ ]:
regressor_df = []
for path in dataset_folders:
    exp = LotrExperiment(path)
    reg_df = exp.motor_regressors
    reg_df["fid"] = exp.dir_name
    reg_df["hdn"] = False
    reg_df.loc[exp.hdn_indexes, "hdn"] = True

    regressor_df.append(reg_df)

regressor_df = pd.concat(regressor_df, ignore_index=True)

In [ ]:
plt.figure(figsize=(3, 3))
plt.scatter(regressor_df["left_1"], regressor_df["right_1"], alpha=0.1, lw=0)
plt.scatter(
    regressor_df.loc[regressor_df["hdn"], "left_1"],
    regressor_df.loc[regressor_df["hdn"], "right_1"],
    alpha=0.1,
    lw=0,
)
plt.axis("equal")

In [ ]:
plt.figure(figsize=(3, 3))
plt.scatter(
    regressor_df.loc[~regressor_df["hdn"], "all_1"],
    regressor_df.loc[~regressor_df["hdn"], "all_1_dfdt"],
    alpha=0.1,
    lw=0,
)
plt.scatter(
    regressor_df.loc[regressor_df["hdn"], "all_1"],
    regressor_df.loc[regressor_df["hdn"], "all_1_dfdt"],
    alpha=0.1,
    lw=0,
)
# plt.axis("equal")

In [ ]:
regressor_df.columns

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(6, 3))
s = 15
for i, coords in enumerate(
    [[regr_df["right_1"], regr_df["left_1"]], [exp.coords[:, 1], exp.coords[:, 2]]]
):
    axs[i].scatter(coords[0], coords[1], c=(0.8,) * 3, s=s)
    axs[i].scatter(
        coords[0][exp.hdn_indexes],
        coords[1][exp.hdn_indexes],
        fc=(0.8,) * 3,
        ec=(0.3,) * 3,
        lw=0.5,
        s=s,
        label="HDNs",
    )

    for k, sel in bout_sel.items():
        axs[i].scatter(coords[0][sel], coords[1][sel], s=s, label=k + "_sel")
    axs[i].axis("equal")


axs[0].plot([0, 0.5], [0, 0.5], "k", lw=0.5)
axs[0].legend(frameon=False, fontsize=7)
axs[0].set(xlabel="Left bouts corr.", ylabel="Right bouts corr.")
axs[1].axis("off")
plt.tight_layout()